In [2]:
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [65]:
def preprocess_and_save_dataset(dataset_path, save_path, max_files_per_class=350, test_size=0.2):
    classes = os.listdir(dataset_path)
    images = []
    labels = []

    subclass_names = []

    for i, class_name in enumerate(classes):
        class_path = os.path.join(dataset_path, class_name)

        if os.path.isdir(class_path):
            for subclass_name in os.listdir(class_path):
                subclass_path = os.path.join(class_path, subclass_name)

                if os.path.isdir(subclass_path):
                    subclass_names.append(subclass_name)

                    count = 0
                    for image_name in os.listdir(subclass_path):
                        if count >= max_files_per_class:
                            break

                        image_path = os.path.join(subclass_path, image_name)
                        image = cv2.imread(image_path)
                        if image is not None:
                            image = cv2.resize(image, (128, 128))
                            images.append(image)
                            labels.append(i)
                            count += 1

    images = np.array(images, dtype=np.float32) / 255.0
    labels = np.array(labels, dtype=np.int32)

    x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=test_size, random_state=42)

    np.savez(save_path, x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)
    print(f"Veri seti başarıyla kaydedildi: {save_path}")

    subclass_names_path = "subclass_names.npy"
    np.save(subclass_names_path, subclass_names)
    print(f"Alt sınıf adları kaydedildi: {subclass_names_path}")

Veri seti başarıyla kaydedildi: atik.npz
Alt sınıf adları kaydedildi: subclass_names.npy


In [66]:
dataset_path = "atiklar"
save_path = "atik.npz"
preprocess_and_save_dataset(dataset_path, save_path, max_files_per_class=350)

In [3]:
data = np.load('atik.npz')

In [4]:
x_train, y_train, x_test, y_test = data['x_train'], data['y_train'], data['x_test'], data['y_test']

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=20,batch_size=64, validation_data=(x_test, y_test))

test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc}")

Epoch 1/20
35/35 [==============================] - 18s 493ms/step - loss: 1.1323 - accuracy: 0.5004 - val_loss: 0.9932 - val_accuracy: 0.4857
Epoch 2/20
35/35 [==============================] - 18s 502ms/step - loss: 0.9030 - accuracy: 0.5670 - val_loss: 0.9301 - val_accuracy: 0.5518
Epoch 3/20
35/35 [==============================] - 18s 523ms/step - loss: 0.8743 - accuracy: 0.5705 - val_loss: 0.8784 - val_accuracy: 0.5911
Epoch 4/20
35/35 [==============================] - 18s 510ms/step - loss: 0.8351 - accuracy: 0.6009 - val_loss: 0.8553 - val_accuracy: 0.6107
Epoch 5/20
35/35 [==============================] - 18s 506ms/step - loss: 0.7795 - accuracy: 0.6482 - val_loss: 0.8670 - val_accuracy: 0.6179
Epoch 6/20
35/35 [==============================] - 19s 533ms/step - loss: 0.7005 - accuracy: 0.6799 - val_loss: 0.8777 - val_accuracy: 0.6179
Epoch 7/20
35/35 [==============================] - 18s 512ms/step - loss: 0.6386 - accuracy: 0.7254 - val_loss: 0.9106 - val_accuracy: 0.6054

In [39]:
test_image_path = "ornek.jpg"
test_image = cv2.imread(test_image_path)

In [40]:
if test_image is not None:
    test_image = cv2.resize(test_image, (128, 128))
    test_image = np.expand_dims(test_image, axis=0) / 255.0
    
    predictions = model.predict(test_image)
    predicted_subclass_index = np.argmax(predictions)

    subclass_names_path = "subclass_names.npy"
    subclass_names = np.load(subclass_names_path)
    
    predicted_subclass_name = subclass_names[predicted_subclass_index]
    print(f"Tahmin edilen alt sınıf: {predicted_subclass_name}")

Tahmin edilen alt sınıf: cam
